# Import Library

In [ ]:
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.metrics import confusion_matrix,accuracy_score
from matplotlib import pyplot
import time
import io

from google.colab import files

KeyboardInterrupt: ignored

# Import Data

In [ ]:
df1 = pd.read_csv("datall.csv").drop(columns=['Unnamed: 0'])
df2 = pd.read_csv("data_tambahan_ok.csv")
df = pd.concat([df1, df2]).reset_index().drop(columns=['index'])
df

# Preprocessing

Mencetak data pertama

In [ ]:
# df.iloc[0]

In [ ]:
# df.info()

Rename kolom paling akhir menjadi label

In [ ]:
df.rename(columns={df.columns[-1]: 'label'}, inplace=True)

In [ ]:
# df.head()

In [ ]:
df.to_excel("pre_encoding_data.xlsx")

Replace Karakter Menjadi Numeric (Sum)


In [ ]:
df = df.replace({'A': '0'}, regex=True) # A: 00
df = df.replace({'T': '1'}, regex=True) # T: 01
df = df.replace({'G': '2'}, regex=True) # G: 10
df = df.replace({'C': '3'}, regex=True) # C: 11

df = df.replace({'hyper': 1}, regex=True)
df = df.replace({'normal': 0}, regex=True)

In [ ]:
df.to_excel("post_encoding_data.xlsx")

In [ ]:
for col in df.columns:
  print(df[col].unique())

In [ ]:
cols = df.columns[:-1]
cols

In [ ]:
for i, data in df.iterrows():
  for col in cols:
    iterable = str(df.loc[i][col])
    df.loc[i, col] = sum(int(x) for x in iterable if x.isdigit())

Menshuffle data

In [ ]:
df = df.reset_index().drop(columns=['index'])

In [ ]:
df.head()

Membagi data menjadi train dan test

In [ ]:
y = df.pop('label')
X = df

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [ ]:
pd.DataFrame(X_train).to_excel('X_train.xlsx')
pd.DataFrame(y_train).to_excel('y_train.xlsx')
pd.DataFrame(X_test).to_excel('X_test.xlsx')
pd.DataFrame(y_test).to_excel('y_test.xlsx')

# Load Existing Data

In [ ]:
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
import joblib
from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.metrics import confusion_matrix,accuracy_score
from matplotlib import pyplot
import time
import io

from google.colab import files

X_train = pd.read_excel('X_train.xlsx').drop(columns=['Unnamed: 0'])
X_test = pd.read_excel('X_test.xlsx').drop(columns=['Unnamed: 0'])
y_train = pd.read_excel('y_train.xlsx').drop(columns=['Unnamed: 0'])
y_test = pd.read_excel('y_test.xlsx').drop(columns=['Unnamed: 0'])

# KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

start_time = time.time()

# KNN = KNeighborsClassifier()
# KNN.fit(X_train, y_train)
# joblib.dump(KNN, "knn.joblib")

print("Train Time: %s seconds" % (time.time() - start_time))
start_time = time.time()

KNN = joblib.load("knn.joblib")
knn_y_pred = KNN.predict(X_test)

print("Test Time: %s seconds" % (time.time() - start_time))

Train Time: 5.14984130859375e-05 seconds
Test Time: 0.011559009552001953 seconds


In [ ]:
KNN.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

In [ ]:
if(len(confusion_matrix(y_test, knn_y_pred).ravel()) > 1):
  tn, fp, fn, tp = confusion_matrix(y_test, knn_y_pred).ravel()
  print(tn, fp, fn, tp)
else:
  knn_cm = confusion_matrix(y_test, knn_y_pred)
  print(knn_cm)
knn_ac = accuracy_score(y_test, knn_y_pred)

6 4 5 21


In [ ]:
knn_ac

0.75

# SVM

In [ ]:
from sklearn.svm import SVC

In [ ]:
start_time = time.time()

# SVM = SVC(gamma='auto')
# SVM.fit(X_train, y_train)
# joblib.dump(SVM, "svm.joblib")

print("Train Time: %s seconds" % (time.time() - start_time))
start_time = time.time()

SVM = joblib.load("svm.joblib")
svm_y_pred = SVM.predict(X_test)

print("Test Time: %s seconds" % (time.time() - start_time))

Train Time: 6.318092346191406e-05 seconds
Test Time: 0.009127616882324219 seconds


In [ ]:
SVM.get_params()

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'auto',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [ ]:
if(len(confusion_matrix(y_test, svm_y_pred).ravel()) > 1):
  tn, fp, fn, tp = confusion_matrix(y_test, svm_y_pred).ravel()
  print(tn, fp, fn, tp)
else:
  svm_cm = confusion_matrix(y_test, svm_y_pred)
  print(svm_cm)
svm_ac = accuracy_score(y_test,svm_y_pred)

7 3 0 26


In [ ]:
svm_ac

0.9166666666666666

# Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
start_time = time.time()

# NB = MultinomialNB()
# NB.fit(X_train, y_train)
# joblib.dump(NB, "nb.joblib")

print("Train Time: %s seconds" % (time.time() - start_time))
start_time = time.time()

NB = joblib.load("nb.joblib")
nb_y_pred = NB.predict(X_test)

print("Test Time: %s seconds" % (time.time() - start_time))

Train Time: 7.581710815429688e-05 seconds
Test Time: 0.005247592926025391 seconds


In [ ]:
NB.get_params()

{'alpha': 1.0, 'class_prior': None, 'fit_prior': True}

In [ ]:
if(len(confusion_matrix(y_test, nb_y_pred).ravel()) > 1):
  tn, fp, fn, tp = confusion_matrix(y_test, nb_y_pred).ravel()
  print(tn, fp, fn, tp)
else:
  nb_cm = confusion_matrix(y_test, nb_y_pred)
  print(nb_cm)
nb_ac = accuracy_score(y_test,nb_y_pred)

4 6 7 19


In [ ]:
nb_ac

0.6388888888888888

# Decision tree

In [ ]:
from sklearn import tree

In [ ]:
start_time = time.time()

# DT = tree.DecisionTreeClassifier()
# DT.fit(X_train, y_train)
# joblib.dump(DT, "dt.joblib")

print("Train Time: %s seconds" % (time.time() - start_time))
start_time = time.time()

DT = joblib.load("dt.joblib")
dt_y_pred = DT.predict(X_test)

print("Test Time: %s seconds" % (time.time() - start_time))

Train Time: 6.794929504394531e-05 seconds
Test Time: 0.00426936149597168 seconds


In [ ]:
DT.get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': None,
 'splitter': 'best'}

In [ ]:
if(len(confusion_matrix(y_test, dt_y_pred).ravel()) > 1):
  tn, fp, fn, tp = confusion_matrix(y_test, dt_y_pred).ravel()
  print(tn, fp, fn, tp)
else:
  dt_cm = confusion_matrix(y_test, dt_y_pred)
  print(dt_cm)
dt_ac = accuracy_score(y_test,dt_y_pred)

9 1 4 22


In [ ]:
dt_ac

0.8611111111111112

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
start_time = time.time()

# RF = RandomForestClassifier()
# RF.fit(X_train, y_train)
# joblib.dump(RF, "rf.joblib")

print("Train Time: %s seconds" % (time.time() - start_time))
start_time = time.time()

RF = joblib.load("rf.joblib")
rf_y_pred = RF.predict(X_test)

print("Test Time: %s seconds" % (time.time() - start_time))

Train Time: 6.29425048828125e-05 seconds
Test Time: 0.06036686897277832 seconds


In [ ]:
RF.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [ ]:
if(len(confusion_matrix(y_test, rf_y_pred).ravel()) > 1):
  tn, fp, fn, tp = confusion_matrix(y_test, rf_y_pred).ravel()
  print(tn, fp, fn, tp)
else:
  rf_cm = confusion_matrix(y_test, rf_y_pred)
  print(rf_cm)
rf_ac = accuracy_score(y_test, rf_y_pred)

9 1 0 26


In [ ]:
rf_ac

0.9722222222222222

# LSTM

In [ ]:
X_train = X_train.to_numpy().reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.to_numpy().reshape((X_test.shape[0], 1, X_test.shape[1]))

In [ ]:
X_train = np.asarray(X_train).astype('float32')
X_test = np.asarray(X_test).astype('float32')

In [ ]:
X_train.shape

(106, 1, 27)

In [ ]:
# # LSTM Model
# model = Sequential()
# model.add(LSTM(32, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2]))) # stacked LSTM
# model.add(Dropout(0.2))
# model.add(LSTM(32, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2]))) # stacked LSTM
# model.add(Dropout(0.2))
# model.add(LSTM(32))
# model.add(Dropout(0.2))
# model.add(Dense(1, activation='sigmoid'))
# model.compile(loss='binary_crossentropy', optimizer='adamax', metrics=['binary_accuracy'])

# start_time = time.time()

# # Early Stopping
# es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)

# # Model Checkpoint
# mc = ModelCheckpoint('best_model.h5', monitor='val_binary_accuracy', mode='max', verbose=1, save_best_only=True)

# history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5000, batch_size=32, callbacks=[es, mc])

# print("Train Time: %s seconds" % (time.time() - start_time))

# traintime = time.time() - start_time

In [ ]:
# # load the saved model
# saved_model = load_model('best_model.h5')


# # evaluate the model
# _, train_acc = model.evaluate(X_train, y_train, verbose=0)
# _, test_acc = model.evaluate(X_test, y_test, verbose=0)
# print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))

In [ ]:
saved_model = load_model('lstmplus.h5')

In [ ]:
# # plot training history
# pyplot.plot(history.history['loss'], label='train')
# pyplot.plot(history.history['val_loss'], label='test')
# pyplot.xlabel('Epoch')
# pyplot.ylabel('Loss')
# pyplot.title("Training History")
# pyplot.legend()
# pyplot.show()

In [ ]:
def _replaceitem(x):
    if x > 0.1:
        return 1
    else:
        return 0

In [ ]:
start_time = time.time()

y_pred = saved_model.predict(X_test)

# print("Train Time: %f seconds" % traintime)
print("Test Time: %s seconds" % (time.time() - start_time))

y_pred = pd.DataFrame({'y_pred': y_pred.round().ravel()})
# y_pred = list(map(_replaceitem, y_pred.ravel()))
results = pd.concat([y_test,y_pred], axis=1)

2/2 [==============================] - 2s 7ms/step
Test Time: 1.8027093410491943 seconds


In [ ]:
if(len(confusion_matrix(y_test, y_pred).ravel()) > 1):
  tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
  print(tn, fp, fn, tp)
else:
  cm = confusion_matrix(y_test, y_pred)
  print(cm)
ac = accuracy_score(y_test,y_pred)

10 0 1 25


In [ ]:
ac

0.9722222222222222

In [ ]:
# from google.colab import drive
# drive.mount('/gdrive')

# model.save('lstmplus.h5')